---
title: "Gamma-Poisson Model"
description: "An extension of the Poisson model for count data that exhibits overdispersion."
categories: [Regression, Count Data, GLM]
image: "Figures/8.png"
order: 10
---

## General Principles 
To model the relationship between a count outcome variable and one or more independent variables with [<span style="color:#0D6EFD">overdispersion 🛈</span>]{#overdispersion}, we can use the _Negative Binomial model_. 

## Considerations
::: callout-caution 
- We have the same considerations as for the [Poisson model](7.&#32;Poisson&#32;model.qmd).
  
- Overdispersion is handled because the Gamma-Poisson model assumes that each Poisson count observation has its own rate. This is an additional parameter specified in the model (in the code, it is `log_days`).
:::
 
## Example
Below is an example code snippet demonstrating a Bayesian Gamma-Poisson model using the Bayesian Inference (BI) package. 

::: {.panel-tabset group="language"}
### Python

In [ ]:
from BI import bi, jnp

# Setup device ------------------------------------------------
m = bi(platform='cpu') # Import

# Import Data & Data Manipulation ------------------------------------------------
# Import
from importlib.resources import files
data_path =  m.load.sim_gamma_poisson(only_path=True)
m.data(data_path, sep=',') 


# Define model ------------------------------------------------
def model(log_days, monastery, y):
    a = m.dist.normal(0, 1, name = 'a', shape=(1,))
    b = m.dist.normal(0, 1, name = 'b', shape=(1,))
    phi = m.dist.exponential(1, name = 'phi', shape=(1,))
    mu = jnp.exp(log_days + a + b * monastery)
    Lambda =  m.dist.gamma(rate = mu*phi, concentration = phi, name = 'Lambda')
    m.dist.poisson(rate = Lambda, obs=y)
# Run MCMC ------------------------------------------------
m.fit(model) # Optimize model parameters through MCMC sampling

# Summary ------------------------------------------------
m.summary() # Get posterior distributions

### R
```R
library(BayesianInference)

# Setup platform------------------------------------------------
m=importBI(platform='cpu')

# Import data ------------------------------------------------
m$data(m$load$sim_gamma_poisson(only_path=T), sep = '')
m$data_to_model(list('log_days', 'monastery', 'y' )) # Send to model (convert to jax array)

# Define model ------------------------------------------------
model <- function(log_days, monastery, y){
  # Parameter prior distributions
  alpha = bi.dist.normal(0, 1, name='alpha', shape=c(1))
  beta = bi.dist.normal(0, 1, name='beta', shape=c(1))
  phi = bi.dist.exponential(1, name='phi', shape=c(1))
  mu = jnp$exp(log_days + alpha + beta * monastery)
  Lambda =  bi.dist.gamma(rate = mu*phi, concentration = phi, name = 'Lambda')
  # Likelihood
  bi.dist.poisson(rate=Lambda, obs=y)
}
# Run MCMC ------------------------------------------------
m$fit(model) # Optimize model parameters through MCMC sampling

# Summary ------------------------------------------------
m$summary() # Get posterior distributions

```

### Julia
```julia
using BayesianInference

# Setup device------------------------------------------------
m = importBI(platform="cpu")

# Import Data & Data Manipulation ------------------------------------------------
# Import
data_path = m.load.sim_gamma_poisson(only_path = true)
m.data(data_path, sep=',')

# Define model ------------------------------------------------
@BI function model(log_days, monastery, y)
    a = m.dist.normal(0, 1, name = "a", shape=(1,))
    b = m.dist.normal(0, 1, name = "b", shape=(1,))
    phi = m.dist.exponential(1, name = "phi", shape=(1,))
    mu = jnp.exp(log_days + a + b * monastery)
    Lambda =  m.dist.gamma(rate = mu*phi, concentration = phi, name = "Lambda")
    m.dist.poisson(rate = Lambda, obs=y)
end

# Run mcmc ------------------------------------------------
m.fit(model)  # Optimize model parameters through MCMC sampling

# Summary ------------------------------------------------
m.summary() # Get posterior distributions
```
:::

## Mathematical Details
### *Bayesian model*
In the Bayesian formulation, we define each parameter with [<span style="color:#0D6EFD">priors 🛈</span>]{#prior}. We can express the Bayesian regression model accounting for prior distributions as follows:

$$
Y_i \sim \text{Poisson}(\lambda_i)
$$

$$
\lambda_i \sim \text{Gamma}(\mu_i \phi, \phi)
$$

$$
\log(\mu_i) = \text{rates}_i + \alpha + \beta X_i
$$

$$
\alpha \sim \text{Normal}(0,1)
$$

$$
\beta \sim \text{Normal}(0,1)
$$

$$
\phi \sim \text{Exponential}(1)
$$

Where:

- $Y_i$ is the dependent variable for observation *i*. 
  
- $\lambda_i$ is the rate parameter of the Poisson distribution for observation *i*, assuming that each Poisson count observation has its own $rate_i$.
  
- $\mu_i$ is the mean rate parameter.
- 
- $\phi$ controls the level of overdispersion in the rates.
  
- $\alpha$ is the intercept term.
  
- $\beta$ is the regression coefficient.
  
- $X_i$ is the value of the predictor variable for observation *i*.


## Notes

::: callout-note 
- We can apply multiple variables similarly as in [chapter 2](2.&#32;Multiple&#32;continuous&#32;variables.qmd).

- We can apply interaction terms similarly as in [chapter 3](3.&#32;Interaction&#32;between&#32;Continuous&#32;Variables.qmd).

- We can apply categorical variables similarly as in [chapter 4](4.&#32;Categorical&#32;variable.qmd).
:::

## Reference(s)